In [1]:
!ls

Spark_sql.ipynb  spark-warehouse  电商推荐.ipynb  推荐概述.ipynb


In [2]:
!hadoop fs -ls /

Found 10 items
-rw-r--r--   1 root supergroup      88444 2021-03-29 21:19 /USA.json
drwxr-xr-x   - root supergroup          0 2021-03-11 21:10 /hbase
-rw-r--r--   1 root supergroup       4662 2021-03-26 20:22 /iris.csv
drwxr-xr-x   - root supergroup          0 2020-12-14 11:59 /output
drwxr-xr-x   - root supergroup          0 2020-12-14 12:08 /output1
drwxr-xr-x   - root supergroup          0 2020-12-14 14:00 /output2
-rw-r--r--   1 root supergroup   22924462 2021-03-30 20:50 /raw_nyc_phil.json
drwx------   - root supergroup          0 2020-12-21 13:00 /tmp
drwxr-xr-x   - root supergroup          0 2020-12-16 11:12 /user
-rw-r--r--   1 root supergroup         42 2020-12-14 11:36 /words


In [3]:
!which python

/miniconda2/envs/py365/bin/python


In [4]:
import os
# 配置spark driver和pyspark运行时，所使用的python解释器路径
PYSPARK_PYTHON = "/miniconda2/envs/py365/bin/python"
JAVA_HOME='/root/bigdata/jdk'
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON
os.environ['JAVA_HOME']=JAVA_HOME
# spark配置信息
from pyspark import SparkConf
from pyspark.sql import SparkSession
#注意先启动standalone模式
# ./start-master.sh -h 192.168.19.137
#  ./start-slave.sh spark://192.168.19.137:7077
SPARK_APP_NAME = "preprocessingBehaviorLog"
SPARK_URL = "spark://192.168.19.137:7077"

conf = SparkConf()    # 创建spark config对象
config = (
	("spark.app.name", SPARK_APP_NAME),    # 设置启动的spark的app名称，没有提供，将随机产生一个名称
	("spark.executor.memory", "6g"),    # 设置该app启动时占用的内存用量，默认1g
	("spark.master", SPARK_URL),    # spark master的地址
    ("spark.executor.cores", "2"),    # 设置spark executor使用的CPU核心数
    # 以下三项配置，可以控制执行器数量
#     ("spark.dynamicAllocation.enabled", True),
#     ("spark.dynamicAllocation.initialExecutors", 1),    # 1个执行器
#     ("spark.shuffle.service.enabled", True)
# 	('spark.sql.pivotMaxValues', '99999'),  # 当需要pivot DF，且值很多时，需要修改，默认是10000
)
# 查看更详细配置及说明：https://spark.apache.org/docs/latest/configuration.html

conf.setAll(config)

# 利用config对象，创建spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [5]:
#通过RDD来创建Datarame
#创建Datarase需要有sparg session
#创刨建RDD蒜要sparkcontext

from pyspark.sql import Row

sc = spark.sparkContext
# spark.conf.set("spark.sql.shuffle.partitions", 6)
# ================直接创建==========================
l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
rdd = sc.parallelize(l)
#为数据添加列名
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
#创建DataFrame
schemaPeople = spark.createDataFrame(people)

In [6]:
schemaPeople

DataFrame[age: bigint, name: string]

In [7]:
schemaPeople.show()

+---+--------+
|age|    name|
+---+--------+
| 25|   Ankit|
| 22|Jalfaizy|
| 20| saurabh|
| 26|    Bala|
+---+--------+



In [8]:
#默认从hadoop的根加载数据
df = spark.read.format("csv").option("header", "true").load("/iris.csv")

In [9]:
df

DataFrame[id: string, SepalLength: string, SepalWidth: string, PetalLength: string, PetalWidth: string, Species: string, cls: string]

In [10]:
df.count()

150

In [11]:
df.columns

['id',
 'SepalLength',
 'SepalWidth',
 'PetalLength',
 'PetalWidth',
 'Species',
 'cls']

In [12]:
# ===============增加一列(或者替换) withColumn===========
#定义一个新的列，数据为其他某列数据的两倍
#如果操作的是原有列，可以替换原有列的数据
df.withColumn('newWidth',df.SepalWidth * 2).show()

+---+-----------+----------+-----------+----------+-------+---+--------+
| id|SepalLength|SepalWidth|PetalLength|PetalWidth|Species|cls|newWidth|
+---+-----------+----------+-----------+----------+-------+---+--------+
|  1|        5.1|       3.5|        1.4|       0.2| setosa|  0|     7.0|
|  2|        4.9|         3|        1.4|       0.2| setosa|  0|     6.0|
|  3|        4.7|       3.2|        1.3|       0.2| setosa|  0|     6.4|
|  4|        4.6|       3.1|        1.5|       0.2| setosa|  0|     6.2|
|  5|          5|       3.6|        1.4|       0.2| setosa|  0|     7.2|
|  6|        5.4|       3.9|        1.7|       0.4| setosa|  0|     7.8|
|  7|        4.6|       3.4|        1.4|       0.3| setosa|  0|     6.8|
|  8|          5|       3.4|        1.5|       0.2| setosa|  0|     6.8|
|  9|        4.4|       2.9|        1.4|       0.2| setosa|  0|     5.8|
| 10|        4.9|       3.1|        1.5|       0.1| setosa|  0|     6.2|
| 11|        5.4|       3.7|        1.5|       0.2|

In [13]:
# ==========删除一列  drop=========================
#删除一列
df.drop('cls').show()

+---+-----------+----------+-----------+----------+-------+
| id|SepalLength|SepalWidth|PetalLength|PetalWidth|Species|
+---+-----------+----------+-----------+----------+-------+
|  1|        5.1|       3.5|        1.4|       0.2| setosa|
|  2|        4.9|         3|        1.4|       0.2| setosa|
|  3|        4.7|       3.2|        1.3|       0.2| setosa|
|  4|        4.6|       3.1|        1.5|       0.2| setosa|
|  5|          5|       3.6|        1.4|       0.2| setosa|
|  6|        5.4|       3.9|        1.7|       0.4| setosa|
|  7|        4.6|       3.4|        1.4|       0.3| setosa|
|  8|          5|       3.4|        1.5|       0.2| setosa|
|  9|        4.4|       2.9|        1.4|       0.2| setosa|
| 10|        4.9|       3.1|        1.5|       0.1| setosa|
| 11|        5.4|       3.7|        1.5|       0.2| setosa|
| 12|        4.8|       3.4|        1.6|       0.2| setosa|
| 13|        4.8|         3|        1.4|       0.1| setosa|
| 14|        4.3|         3|        1.1|

In [14]:
#================ 统计信息 describe================
df.describe().show()
#计算某一列的描述信息
df.describe('cls').show()   

+-------+------------------+------------------+-------------------+------------------+------------------+---------+------------------+
|summary|                id|       SepalLength|         SepalWidth|       PetalLength|        PetalWidth|  Species|               cls|
+-------+------------------+------------------+-------------------+------------------+------------------+---------+------------------+
|  count|               150|               150|                150|               150|               150|      150|               150|
|   mean|              75.5| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|               1.0|
| stddev|43.445367992456916|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|0.8192319205190406|
|    min|                 1|               4.3|                  2|                 1|               0.1|   setosa|                 0|
|    max|                99|               7.9|        

In [15]:
# ===============提取部分列 select==============
df.select('SepalLength','SepalWidth').show()

+-----------+----------+
|SepalLength|SepalWidth|
+-----------+----------+
|        5.1|       3.5|
|        4.9|         3|
|        4.7|       3.2|
|        4.6|       3.1|
|          5|       3.6|
|        5.4|       3.9|
|        4.6|       3.4|
|          5|       3.4|
|        4.4|       2.9|
|        4.9|       3.1|
|        5.4|       3.7|
|        4.8|       3.4|
|        4.8|         3|
|        4.3|         3|
|        5.8|         4|
|        5.7|       4.4|
|        5.4|       3.9|
|        5.1|       3.5|
|        5.7|       3.8|
|        5.1|       3.8|
+-----------+----------+
only showing top 20 rows



In [16]:
# ==================基本统计功能 distinct count=====
df.select('cls').distinct().count()

3

In [17]:
# 分组统计 groupby(colname).agg({'col':'fun','col2':'fun2'})
df.groupby('cls').agg({'SepalWidth':'mean','SepalLength':'max'}).show()

# avg(), count(), countDistinct(), first(), kurtosis(),
# max(), mean(), min(), skewness(), stddev(), stddev_pop(),
# stddev_samp(), sum(), sumDistinct(), var_pop(), var_samp() and variance()

+---+----------------+------------------+
|cls|max(SepalLength)|   avg(SepalWidth)|
+---+----------------+------------------+
|  0|             5.8| 3.428000000000001|
|  1|               7|2.7700000000000005|
|  2|             7.9|2.9739999999999998|
+---+----------------+------------------+



In [18]:
# 自定义的汇总方法
import pyspark.sql.functions as fn
#调用函数并起一个别名,fn内部有哪些函数可以在pycharm联想
df.agg(fn.count('SepalWidth').alias('width_count'),fn.countDistinct('cls').alias('distinct_cls_count')).show()

+-----------+------------------+
|width_count|distinct_cls_count|
+-----------+------------------+
|        150|                 3|
+-----------+------------------+



In [19]:
#====================数据集拆成两部分 randomSplit ===========
#设置数据比例将数据划分为两部分
trainDF, testDF = df.randomSplit([0.6, 0.4])

In [20]:
# ================采样数据 sample===========
#withReplacement：是否有放回的采样
#fraction：采样比例
#seed：随机种子
sdf = df.sample(False,0.2,100)

In [21]:
#查看两个数据集在类别上的差异 subtract，确保训练数据集覆盖了所有分类
diff_in_train_test = testDF.select('cls').subtract(trainDF.select('cls'))
diff_in_train_test.distinct().count()

0

In [22]:
# ================交叉表 crosstab=============
# 一行代表一个类别，这个类别中sepallength的分布式怎么样的
df.crosstab('cls','SepalLength').show()

+---------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|cls_SepalLength|4.3|4.4|4.5|4.6|4.7|4.8|4.9|  5|5.1|5.2|5.3|5.4|5.5|5.6|5.7|5.8|5.9|  6|6.1|6.2|6.3|6.4|6.5|6.6|6.7|6.8|6.9|  7|7.1|7.2|7.3|7.4|7.6|7.7|7.9|
+---------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|              2|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  1|  1|  3|  1|  2|  2|  2|  6|  5|  4|  0|  5|  2|  3|  0|  1|  3|  1|  1|  1|  4|  1|
|              1|  0|  0|  0|  0|  0|  0|  1|  2|  1|  1|  0|  1|  5|  5|  5|  3|  2|  4|  4|  2|  3|  2|  1|  2|  3|  1|  1|  1|  0|  0|  0|  0|  0|  0|  0|
|              0|  1|  3|  1|  4|  2|  5|  4|  8|  8|  3|  1|  5|  2|  0|  2|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
+---------------+---+---+---+---+---+---+---+---+---

In [23]:
#================== 综合案例 + udf================
# 测试数据集中有些类别在训练集中是不存在的，找到这些数据集做后续处理
trainDF,testDF = df.randomSplit([0.99,0.01])

diff_in_train_test = trainDF.select('cls').subtract(testDF.select('cls')).distinct().show()

#首先找到这些类，整理到一个列表
not_exist_cls = trainDF.select('cls').subtract(testDF.select('cls')).distinct().rdd.map(lambda x :x[0]).collect()
print(not_exist_cls)
#定义一个方法，用于检测
def should_remove(x):
    if x in not_exist_cls:
        return -1
    else :
        return x

#创建udf，udf函数需要两个参数：
# Function
# Return type (in my case StringType())

#在RDD中可以直接定义函数，交给rdd的transformatioins方法进行执行
#在DataFrame中需要通过udf将自定义函数封装成udf函数再交给DataFrame进行调用执行

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

#这一步是封装
check = udf(should_remove,StringType())

resultDF = trainDF.withColumn('New_cls',check(trainDF['cls'])).filter('New_cls <> -1')

resultDF.show()

+---+
|cls|
+---+
|  0|
|  1|
+---+

['0', '1']
+---+-----------+----------+-----------+----------+---------+---+-------+
| id|SepalLength|SepalWidth|PetalLength|PetalWidth|  Species|cls|New_cls|
+---+-----------+----------+-----------+----------+---------+---+-------+
|101|        6.3|       3.3|          6|       2.5|virginica|  2|      2|
|102|        5.8|       2.7|        5.1|       1.9|virginica|  2|      2|
|103|        7.1|         3|        5.9|       2.1|virginica|  2|      2|
|104|        6.3|       2.9|        5.6|       1.8|virginica|  2|      2|
|105|        6.5|         3|        5.8|       2.2|virginica|  2|      2|
|106|        7.6|         3|        6.6|       2.1|virginica|  2|      2|
|107|        4.9|       2.5|        4.5|       1.7|virginica|  2|      2|
|108|        7.3|       2.9|        6.3|       1.8|virginica|  2|      2|
|109|        6.7|       2.5|        5.8|       1.8|virginica|  2|      2|
|110|        7.2|       3.6|        6.1|       2.5|virginica|  2

# JSON数据的处理

In [24]:
jsonString = [
"""{ "id" : "01001", "city" : "AGAWAM",  "pop" : 15338, "state" : "MA" }""",
"""{ "id" : "01002", "city" : "CUSHMAN", "pop" : 36963, "state" : "MA" }"""
]
jsonRDD = sc.parallelize(jsonString)   # stringJSONRDD
jsonDF =  spark.read.json(jsonRDD)  # convert RDD into DataFrame


In [25]:
#spark处理json会做类型推断
jsonDF.printSchema()

root
 |-- city: string (nullable = true)
 |-- id: string (nullable = true)
 |-- pop: long (nullable = true)
 |-- state: string (nullable = true)



In [26]:
jsonDF.show()

+-------+-----+-----+-----+
|   city|   id|  pop|state|
+-------+-----+-----+-----+
| AGAWAM|01001|15338|   MA|
|CUSHMAN|01002|36963|   MA|
+-------+-----+-----+-----+



In [27]:
jsonDF = spark.read.json("/USA.json")

In [28]:
# truncate=False
jsonDF.show()

+--------------------+--------------------+----+--------------------+-------+
|     _corrupt_record|            geometry|  id|          properties|   type|
+--------------------+--------------------+----+--------------------+-------+
|{"type":"FeatureC...|                null|null|                null|   null|
|                null|[WrappedArray(Wra...|  01|           [Alabama]|Feature|
|                null|[WrappedArray(Wra...|  02|            [Alaska]|Feature|
|                null|[WrappedArray(Wra...|  04|           [Arizona]|Feature|
|                null|[WrappedArray(Wra...|  05|          [Arkansas]|Feature|
|                null|[WrappedArray(Wra...|  06|        [California]|Feature|
|                null|[WrappedArray(Wra...|  08|          [Colorado]|Feature|
|                null|[WrappedArray(Wra...|  09|       [Connecticut]|Feature|
|                null|[WrappedArray(Wra...|  10|          [Delaware]|Feature|
|                null|[WrappedArray(Wra...|  11|[District of Col

In [29]:
#去看每一部分的类型
jsonDF.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- geometry: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |-- type: string (nullable = true)



In [30]:
jsonDF.filter(jsonDF.id>40).show(10)

+---------------+--------------------+---+----------------+-------+
|_corrupt_record|            geometry| id|      properties|   type|
+---------------+--------------------+---+----------------+-------+
|           null|[WrappedArray(Wra...| 41|        [Oregon]|Feature|
|           null|[WrappedArray(Wra...| 42|  [Pennsylvania]|Feature|
|           null|[WrappedArray(Wra...| 44|  [Rhode Island]|Feature|
|           null|[WrappedArray(Wra...| 45|[South Carolina]|Feature|
|           null|[WrappedArray(Wra...| 46|  [South Dakota]|Feature|
|           null|[WrappedArray(Wra...| 47|     [Tennessee]|Feature|
|           null|[WrappedArray(Wra...| 48|         [Texas]|Feature|
|           null|[WrappedArray(Wra...| 49|          [Utah]|Feature|
|           null|[WrappedArray(Wra...| 50|       [Vermont]|Feature|
|           null|[WrappedArray(Wra...| 51|      [Virginia]|Feature|
+---------------+--------------------+---+----------------+-------+
only showing top 10 rows



In [31]:
jsonDF.createOrReplaceTempView("tmp_table")

In [32]:
resultDF = spark.sql("select * from tmp_table where id>40")
resultDF.show(10)

+---------------+--------------------+---+----------------+-------+
|_corrupt_record|            geometry| id|      properties|   type|
+---------------+--------------------+---+----------------+-------+
|           null|[WrappedArray(Wra...| 41|        [Oregon]|Feature|
|           null|[WrappedArray(Wra...| 42|  [Pennsylvania]|Feature|
|           null|[WrappedArray(Wra...| 44|  [Rhode Island]|Feature|
|           null|[WrappedArray(Wra...| 45|[South Carolina]|Feature|
|           null|[WrappedArray(Wra...| 46|  [South Dakota]|Feature|
|           null|[WrappedArray(Wra...| 47|     [Tennessee]|Feature|
|           null|[WrappedArray(Wra...| 48|         [Texas]|Feature|
|           null|[WrappedArray(Wra...| 49|          [Utah]|Feature|
|           null|[WrappedArray(Wra...| 50|       [Vermont]|Feature|
|           null|[WrappedArray(Wra...| 51|      [Virginia]|Feature|
+---------------+--------------------+---+----------------+-------+
only showing top 10 rows



In [33]:
jsonString = [
"""{ "id" : "01001", "city" : "AGAWAM",  "pop" : 15338, "state" : "MA" }""",
"""{ "id" : "01002", "city" : "CUSHMAN", "pop" : 36963, "state" : "MA" }"""
]

jsonRDD = sc.parallelize(jsonString)

from pyspark.sql.types import *

#定义结构类型
#StructType：schema的整体结构，表示JSON的对象结构
#XXXStype:指的是某一列的数据类型
jsonSchema = StructType() \
  .add("id", StringType(),True) \
  .add("city", StringType()) \
  .add("pop" , LongType()) \
  .add("state",StringType())

# 如果类型不对，就会为null
# jsonSchema = StructType() \
#   .add("id", LongType(),True) \
#   .add("city", StringType()) \
#   .add("pop" , DoubleType()) \
#   .add("state",StringType())

reader = spark.read.schema(jsonSchema)

jsonDF = reader.json(jsonRDD)
jsonDF.printSchema()
jsonDF.show()

root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- pop: long (nullable = true)
 |-- state: string (nullable = true)

+-----+-------+-----+-----+
|   id|   city|  pop|state|
+-----+-------+-----+-----+
|01001| AGAWAM|15338|   MA|
|01002|CUSHMAN|36963|   MA|
+-----+-------+-----+-----+



In [34]:
'''
1.删除重复数据

groupby().count()：可以看到数据的重复情况
'''
df = spark.createDataFrame([
  (1, 144.5, 5.9, 33, 'M'),
  (2, 167.2, 5.4, 45, 'M'),
  (3, 124.1, 5.2, 23, 'F'),
  (4, 144.5, 5.9, 33, 'M'),
  (5, 133.2, 5.7, 54, 'F'),
  (3, 124.1, 5.2, 23, 'F'),
  (5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])

# 查看重复记录
#无意义重复数据去重：数据中行与行完全重复
# 1.首先删除完全一样的记录,变为6条了
df2 = df.dropDuplicates()
df2.show()


+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [35]:
#有意义去重：删除除去无意义字段之外的完全重复的行数据
# 2.其次，关键字段值完全一模一样的记录（在这个例子中，是指除了id之外的列一模一样）
# 删除某些字段值完全一样的重复记录，subset参数定义这些字段
df3 = df2.dropDuplicates(subset = [c for c in df2.columns if c!='id'])
df3.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [36]:
# 3.有意义的重复记录去重之后，再看某个无意义字段的值是否有重复（在这个例子中，是看id是否重复）
# 查看某一列是否有重复值，就是把id取出来，在distinct id取出来，对比
import pyspark.sql.functions as fn
df3.agg(fn.count('id').alias('id_count'),fn.countDistinct('id').alias('distinct_id_count')).collect()

[Row(id_count=5, distinct_id_count=4)]

In [37]:
# 4.对于id这种无意义的列重复，添加另外一列自增id，仅仅是增加一个无意义的自增id而已
df3.withColumn('new_id',fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



In [38]:
'''
2.处理缺失值
2.1 对缺失值进行删除操作(行，列)
2.2 对缺失值进行填充操作(列的均值)
2.3 对缺失值对应的行或列进行标记
'''
df_miss = spark.createDataFrame([
(1, 143.5, 5.6, 28,'M', 100000),
(2, 167.2, 5.4, 45,'M', None),
(3, None , 5.2, None, None, None),
(4, 144.5, 5.9, 33, 'M', None),
(5, 133.2, 5.7, 54, 'F', None),
(6, 124.1, 5.2, None, 'F', None),
(7, 129.2, 5.3, 42, 'M', 76000),],
 ['id', 'weight', 'height', 'age', 'gender', 'income'])

# 1.计算每条记录的缺失值情况

df_miss.rdd.map(lambda row:(row['id'],sum([c==None for c in row]))).collect()
# [(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]



[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [39]:
# 2.计算各列的缺失情况百分比  fn.count(c)这一列多少个有值，fn.count(c)这个c为None不会记上的
df_miss.agg(*[(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing') for c in df_miss.columns]).show()



+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



In [40]:
# 3、删除缺失值过于严重的列
# 其实是先建一个DF，不要缺失值的列
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])

df_miss_no_income

DataFrame[id: bigint, weight: double, height: double, age: bigint, gender: string]

In [42]:
# 4、按照缺失值删除行（threshold是根据一行记录中，缺失字段的百分比的定义,一行有3个缺少，代表要删除）
df_miss_no_income.dropna(thresh=3).show()

# 5、填充缺失值，可以用fillna来填充缺失值，
# 对于bool类型、或者分类类型，可以为缺失值单独设置一个类型，missing
# 对于数值类型，可以用均值或者中位数等填充

# fillna可以接收两种类型的参数：
# 一个数字、字符串，这时整个DataSet中所有的缺失值都会被填充为相同的值。
# 也可以接收一个字典｛列名：值｝这样

# 先计算均值
means = df_miss_no_income.agg( *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+

+---+------------------+-----------------+----+
| id|            weight|           height| age|
+---+------------------+-----------------+----+
|4.0|140.28333333333333|5.471428571428571|40.4|
+---+------------------+-----------------+----+



In [45]:
import pandas
# 先计算均值，并组织成一个字典
means = df_miss_no_income.agg( *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']).toPandas().to_dict('records')[0]
means

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4}

In [46]:
# 然后添加其它的列
means['gender'] = 'missing'

df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



In [50]:
'''
3、异常值处理
异常值：不属于正常的值 包含：缺失值，超过正常范围内的较大值或较小值
分位数去极值
中位数绝对偏差去极值
正态分布去极值
上述三种操作的核心都是：通过原始数据设定一个正常的范围，超过此范围的就是一个异常值
'''
df_outliers = spark.createDataFrame([
(1, 143.5, 5.3, 28),
(2, 154.2, 5.5, 45),
(3, 342.3, 5.1, 99),
(4, 144.5, 5.5, 33),
(5, 133.2, 5.4, 54),
(6, 124.1, 5.1, 21),
(7, 129.2, 5.3, 42),
], ['id', 'weight', 'height', 'age'])
# 设定范围 超出这个范围的 用边界值替换

# approxQuantile方法接收三个参数：参数1，列名；参数2：想要计算的分位点，可以是一个点，也可以是一个列表（0和1之间的小数），第三个参数是能容忍的误差，如果是0，代表百分百精确计算。

cols = ['weight', 'height', 'age']

bounds = {}
for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
        ]

print(bounds)
# {'age': [-11.0, 93.0], 'height': [4.499999999999999, 6.1000000000000005], 'weight': [91.69999999999999, 191.7]}

# 为异常值字段打标志,如果小于下线，或者大于上线
outliers = df_outliers.select(*['id'] + [( (df_outliers[c] < bounds[c][0]) | (df_outliers[c] > bounds[c][1]) ).alias(c + '_o') for c in cols ])
outliers.show()
#
# +---+--------+--------+-----+
# | id|weight_o|height_o|age_o|
# +---+--------+--------+-----+
# |  1|   false|   false|false|
# |  2|   false|   false|false|
# |  3|    true|   false| true|
# |  4|   false|   false|false|
# |  5|   false|   false|false|
# |  6|   false|   false|false|
# |  7|   false|   false|false|
# +---+--------+--------+-----+



{'weight': [91.69999999999999, 191.7], 'height': [4.499999999999999, 6.1000000000000005], 'age': [-11.0, 93.0]}
+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+



In [51]:
# 再回头看看这些异常值的值，重新和原始数据关联

df_outliers = df_outliers.join(outliers, on='id')
df_outliers.filter('weight_o').select('id', 'weight').show()
# +---+------+
# | id|weight|
# +---+------+
# |  3| 342.3|
# +---+------+



+---+------+
| id|weight|
+---+------+
|  3| 342.3|
+---+------+



In [52]:
#这个是年龄的
df_outliers.filter('age_o').select('id', 'age').show()
# +---+---+
# | id|age|
# +---+---+
# |  3| 99|
# +---+---+

+---+---+
| id|age|
+---+---+
|  3| 99|
+---+---+

